In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import urllib.parse
import seaborn as sns
from tqdm import tqdm

pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

In [52]:
# Load the TSV file, ignoring lines that start with '#'
articles = pd.read_csv('data/wikispeedia_paths-and-graph/articles.tsv', sep='\t', comment='#')
categories = pd.read_csv('data/wikispeedia_paths-and-graph/categories.tsv', sep='\t', comment='#')
links = pd.read_csv('data/wikispeedia_paths-and-graph/links.tsv', sep='\t', comment='#')
paths_finished = pd.read_csv('data/wikispeedia_paths-and-graph/paths_finished.tsv', sep='\t', comment='#')
paths_unfinished = pd.read_csv('data/wikispeedia_paths-and-graph/paths_unfinished.tsv', sep='\t', comment='#')


In [53]:

def load_distance_matrix(filepath):
    # Read the file and create matrix
    matrix = []
    with open(filepath, 'r') as file:
        for line in file:
            # Skip comment lines and empty lines
            if line.startswith('#') or not line.strip():
                continue
            # Convert each character to integer, ignoring underscores
            row = [int(char.replace('_', '-1')) for char in line.strip()]
            matrix.append(row)
    
    return np.array(matrix)

filepath = "data/wikispeedia_paths-and-graph/shortest-path-distance-matrix.txt"
distance_matrix = load_distance_matrix(filepath)



In [54]:
articles.columns = ['article']
articles['article'] = articles['article'].apply(lambda x: urllib.parse.unquote(x).replace('_', ' '))
categories.columns =['article', 'category']
categories['category'] = categories['category'].str.replace(r'^subject\.', '', regex=True)
links.columns = ['linkSource', 'linkTarget']
paths_finished.columns = ['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating']
paths_unfinished.columns = ['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'target', 'type']
paths_unfinished['target'] = paths_unfinished['target'].apply(lambda x: urllib.parse.unquote(x).replace('_', ' '))

In [55]:
paths_finished['status'] = 'finished'
paths_unfinished['status'] = 'unfinished'

# Concatenate the dataframes with the added 'status' column
concatenated_df = pd.concat(
    [
        paths_finished[['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating', 'status']],
        paths_unfinished[['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'target', 'type', 'status']]
    ],
    ignore_index=True
)

In [ ]:
# Create a copy of concatenated_df to avoid modifying the original DataFrame
clean_merge = concatenated_df.copy()

# Modify 'durationInSec' in the new DataFrame based on the condition
for index, row in clean_merge.iterrows():
    if row['type'] == 'timeout':  # Check if the last element in 'path' is 'timeout'
        clean_merge.loc[index, 'durationInSec'] -= 1800  # Subtract 1800 from 'durationInSec'

def find_backtracks(path):
    backtracks = []
    words = path.split(';')
    stack = []

    for word in words:
        if word == "<":
            if stack:
                backtracks.append(stack.pop())
        else:
            url_decoded = urllib.parse.unquote(word).replace('_', ' ')
            stack.append(url_decoded)

    return backtracks

paths_finished['backtracks'] = paths_finished['path'].apply(find_backtracks)
paths_unfinished['backtracks'] = paths_unfinished['path'].apply(find_backtracks)
clean_merge['backtracks'] = clean_merge['path'].apply(find_backtracks)

# Frustration

In [ ]:
# Collect ratings for paths with and without backtracking
filtered_ratings_back = []
filtered_ratings = []

for index, row in paths_finished.iterrows():
    if '<' in row['path'].split(';') and pd.notna(row['rating']):
        filtered_ratings_back.append(row['rating'])  # Paths with backtracking
    elif '<' not in row['path'].split(';') and pd.notna(row['rating']):
        filtered_ratings.append(row['rating'])  # Paths without backtracking

# Calculate the average for each group
average_back = sum(filtered_ratings_back) / len(filtered_ratings_back) if filtered_ratings_back else 0
average_no_back = sum(filtered_ratings) / len(filtered_ratings) if filtered_ratings else 0

print("Average rating (with backtracking):", average_back)
print("Average rating (without backtracking):", average_no_back)

# Plot the histograms
plt.figure(figsize=(10, 6))

# Plot histogram for paths with backtracking
plt.hist(filtered_ratings_back, bins=5, color='orange', edgecolor='black', alpha=0.7, label='With Backtracking')

# Plot histogram for paths without backtracking
plt.hist(filtered_ratings, bins=5, color='skyblue', edgecolor='black', alpha=0.7, label='Without Backtracking')

# Add vertical lines for the averages
plt.axvline(x=average_back, color='red', linestyle='-', linewidth=2, label=f'Average (with backtracking) = {average_back:.2f}')
plt.axvline(x=average_no_back, color='blue', linestyle='-', linewidth=2, label=f'Average (without backtracking) = {average_no_back:.2f}')

# Add labels and title
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.title('Histogram of Ratings for Paths With and Without Backtracking')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()


In [ ]:
print(f"Across the study, we have {len(paths_finished)} paths, while {len(paths_unfinished)} remain unfinished, for a total of {len(clean_merge)} paths.")

In [ ]:
# Calculate the average number of backtracks for finished vs unfinished paths
avg_backtracks_finished = clean_merge[clean_merge['status'] == 'finished']['backtracks'].apply(len).mean()
avg_backtracks_unfinished = clean_merge[clean_merge['status'] != 'finished']['backtracks'].apply(len).mean()

# Output the results
print("\nAverage number of backtracks per game:")
print(f"Finished paths: {avg_backtracks_finished:.2f}")
print(f"Unfinished paths: {avg_backtracks_unfinished:.2f}")


In [ ]:
# Get finished games with and without backtracks

finished_with_bt = paths_finished[paths_finished['backtracks'].apply(len) > 0]
finished_no_bt = paths_finished[paths_finished['backtracks'].apply(len) == 0]

# Calculate average duration for each group
avg_duration_with_bt = finished_with_bt['durationInSec'].median()
avg_duration_no_bt = finished_no_bt['durationInSec'].median()

# Group finished games by number of backtracks and calculate median duration
backtrack_counts = paths_finished.groupby(paths_finished['backtracks'].apply(len)).size()
backtrack_durations = paths_finished.groupby(paths_finished['backtracks'].apply(len))['durationInSec'].median()

# Filter to keep only counts with at least 10 samples
filtered_counts = backtrack_counts[backtrack_counts >= 10]
filtered_durations = backtrack_durations[filtered_counts.index]

# Create the plot
plt.figure(figsize=(10,6))
plt.plot(filtered_durations.index, filtered_durations.values, marker='o')
plt.axvline(x=avg_backtracks_finished, color='r', linestyle='--', label='Average backtracks')
plt.axhline(y=avg_duration_with_bt, color='g', linestyle='--', label='Average duration')
plt.xlabel('Number of Backtracks')
plt.ylabel('Median Duration (seconds)')
plt.title('Median Game Duration vs Number of Backtracks (10+ samples)')
plt.grid(True)

# Add a trend line
z = np.polyfit(filtered_durations.index, filtered_durations.values, 1)
p = np.poly1d(z)
plt.plot(filtered_durations.index, p(filtered_durations.index), "r--", alpha=0.8, 
         label=f'Trend line (slope: {z[0]:.1f})')

# Calculate R-squared
r_squared = r2_score(filtered_durations.values, p(filtered_durations.index))

plt.legend()
plt.text(0.05, 0.95, f'R-squared: {r_squared:.4f}', transform=plt.gca().transAxes, 
         verticalalignment='top')
plt.show()

print("Counts of games with each number of backtracks:")
print(filtered_counts)
print(f"\nR-squared value: {r_squared:.4f}")



In [ ]:
# Group finished games by number of backtracks and calculate median duration
backtrack_counts = paths_finished.groupby(paths_finished['backtracks'].apply(len)).size()
backtrack_ratings = paths_finished.groupby(paths_finished['backtracks'].apply(len))['rating'].mean()

# Filter to keep only counts with at least 10 samples
filtered_counts = backtrack_counts[backtrack_counts >= 10]
filtered_ratings = backtrack_ratings[filtered_counts.index]

# Create the plot
plt.figure(figsize=(10,6))
plt.plot(filtered_ratings.index, filtered_ratings.values, marker='o')
plt.xlabel('Number of Backtracks')
plt.ylabel('Average Rating')
plt.axvline(x=avg_backtracks_finished, color='r', linestyle='--', label='Average backtracks')
plt.axhline(y=paths_finished['rating'].mean(), color='g', linestyle='--', label='Average rating')
plt.title('Average Rating vs Number of Backtracks (10+ samples)')
plt.grid(True)

# Add a trend line
z = np.polyfit(filtered_ratings.index, filtered_ratings.values, 1)
p = np.poly1d(z)

plt.plot(filtered_ratings.index, p(filtered_ratings.index), "r--", alpha=0.8, 
         label=f'Trend line (slope: {z[0]:.1f})')

# Calculate R-squared
r_squared = r2_score(filtered_ratings.values, p(filtered_ratings.index))

plt.legend()
plt.text(0.05, 0.95, f'R-squared: {r_squared:.4f}', transform=plt.gca().transAxes, 
         verticalalignment='top')
plt.show()

print("Counts of games with each number of backtracks:")
print(filtered_counts)
print(f"\nR-squared value: {r_squared:.4f}")

In [ ]:

# Create a DataFrame with counts of backtracks and traversed
backtracks = pd.Series(clean_merge['backtracks'].explode().dropna().tolist())
def path_to_list(path):
    return [urllib.parse.unquote(word).replace('_', ' ') for word in path.replace('<;', '').split(';')]

clean_merge["traversed"] = clean_merge["path"].apply(path_to_list)
traversed = pd.Series(clean_merge['traversed'].explode().dropna().tolist())

word_counts = pd.DataFrame({
    'backtracks': backtracks.value_counts(),
    'traversed': traversed.value_counts()
})

# Calculate the ratio of backtracks to traversed
word_counts['backtrack_ratio'] = word_counts['backtracks'] / word_counts['traversed']

# Fill NaN values with 0 (for words that were traversed but never backtracked)
word_counts['backtrack_ratio'] = word_counts['backtrack_ratio'].fillna(0)

# Sort by the ratio in descending order
word_counts_sorted = word_counts.sort_values('backtrack_ratio', ascending=False)

# Display the top 20 words with highest backtrack ratios for words with at least 100 occurrences
print(word_counts_sorted[word_counts_sorted['traversed'] >= 0].head(20))

# Optional: Plot the distribution of backtrack ratios
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.hist(word_counts['backtrack_ratio'], bins=50, edgecolor='black')
plt.title('Distribution of Backtrack Ratios')
plt.xlabel('Backtrack Ratio')
plt.ylabel('Frequency')
plt.show()



In [ ]:
paths_unfinished.groupby('type').count()

In [ ]:
# Count the number of games in each 'type' group
game_counts_by_type = paths_unfinished.groupby('type').size()

# Display the result
print(game_counts_by_type)


# Explanation

In [ ]:
# Get all backtrack targets
backtrack_targets = clean_merge[clean_merge['backtracks'].apply(len) > 0][['backtracks','target']]
print(backtrack_targets.head())
# Create a list to store individual backtrack entries
backtrack_entries = []

# Iterate through each row that has backtracks
for _, row in backtrack_targets.iterrows():
    # For each backtrack in the row
    for backtrack in row['backtracks']:
        # Add an entry with the backtrack and target
        backtrack_entries.append({
            'backtrack': backtrack,
            'target': row['target']
        })

# Convert to DataFrame
backtrack_df = pd.DataFrame(backtrack_entries)

print("\nArticle of backtrack vs target")
print(backtrack_df.value_counts().head(20))


In [ ]:
# Find missing targets more efficiently and store them in a list
articles_set = set(articles['article'])
missing_targets = [target for target in paths_unfinished['target'] if target not in articles_set]

# Print missing targets if any
if missing_targets:
    print("Missing targets:")
    for target in missing_targets:
        print(target)
else:
    print("No missing targets found.")


# Solution

In [ ]:
all_path_similarities = pd.read_csv('data/similarities.csv')
all_path_similarities['last_article'] = all_path_similarities['path'].apply(lambda x: eval(x)[0][-1])
all_path_similarities_unfinished = all_path_similarities[all_path_similarities['last_article'] != all_path_similarities['target']]
all_path_similarities_finished = all_path_similarities[all_path_similarities['last_article'] == all_path_similarities['target']]
all_path_similarities_finished.head()

In [ ]:
# Take only the last 10 elements of each path
def shorten_list(x,n=10):
    x = eval(x.replace(" ", ""))  # Convert string representation of list to actual list
    x=x[0] #unpack the list
    if len(x) > n:
        return x[-n:]
    return x

def handle_unfinished(entry,n=9):
    similarities = eval(entry['OpenAI'].replace(" ", ""))  # Convert string representation of list to actual list
    path = eval(entry['path'])
    similarities = similarities[0] #unpack the list
    path = path[0]

    max_value = max(similarities)
    if max_value < 0.3:
        return [],[]
    max_index = similarities.index(max_value)
    similarities = similarities[:max_index + 1]  # Keep only values up to and including max
    path = path[:max_index + 1]
    if len(similarities) > n:
        return similarities[-n:],path[-n:]
    return similarities,path

all_path_similarities_unfinished_copy = all_path_similarities_unfinished.copy()
all_path_similarities_unfinished_copy['OpenAI'],all_path_similarities_unfinished_copy['path'] = zip(*all_path_similarities_unfinished_copy.apply(handle_unfinished, axis=1))

similarities_unfinished = all_path_similarities_unfinished_copy[all_path_similarities_unfinished_copy['OpenAI'].apply(lambda x: len(x) > 0)][['OpenAI','path','target']]
print(similarities_unfinished.head())


all_path_similarities_finished_copy = all_path_similarities_finished.copy()
all_path_similarities_finished_copy['OpenAI'] = all_path_similarities_finished_copy['OpenAI'].apply(shorten_list)
similarities_finished = all_path_similarities_finished_copy['OpenAI']

In [ ]:
def get_missing_links(entry):
    # Return a Series instead of a tuple
    return pd.Series({
        'biggest_similarity_article': entry['path'][-1],
        'target': entry['target']
    })

unfinished_missing_links = similarities_unfinished.apply(get_missing_links, axis=1)
print("Most common pairs of articles that may be missing links:")
print(unfinished_missing_links.value_counts().head(20))


In [ ]:
final_results = all_path_similarities
# Create a figure with subplots for the first 10 paths
fig, axes = plt.subplots(5, 2, figsize=(15, 25))
axes = axes.flatten()

# Colors for each model
colors = {
    'OpenAI': 'green'
}

# Plot first 10 paths
for i in range(10):
    path = final_results.apply(lambda x: eval(x['path']),axis=1).iloc[i][0]
    target = final_results['target'].iloc[i]
    
    # Plot each model's similarities
    for model in ['OpenAI']:
        similarities = final_results.apply(lambda x: eval(x[model]),axis=1).iloc[i][0]
        axes[i].plot(range(len(similarities)), similarities, 
                    marker='o', label=model, color=colors[model])
    
    # Customize the plot
    axes[i].set_title(f'Path {i+1}\nTarget: {target}')
    axes[i].set_xlabel('Step in Path')
    axes[i].set_ylabel('Cosine Similarity')
    axes[i].set_ylim(0, 1)
    axes[i].grid(True)
    axes[i].legend()
    
    # Add path words as x-tick labels
    axes[i].set_xticks(range(len(path)))
    axes[i].set_xticklabels(path, rotation=45, ha='right')

plt.tight_layout()
plt.show()


In [ ]:
# Create figure and axis
plt.figure(figsize=(15, 8))
n=10

# Plot first 100 similarity evolutions
for i in range(1000):
    similarities = all_path_similarities_finished_copy['OpenAI'].iloc[i]
    reversed_similarities = similarities[::-1]
    indexes = range(n,n-len(reversed_similarities),-1)
    plt.plot(indexes, reversed_similarities)


plt.xlabel('distance to end of game')
plt.ylabel('Similarity Score') 
plt.title('Evolution of Similarities for First 100 Paths')

# Add grid for better readability
plt.grid(True)

plt.show()


In [ ]:
distance_to_series_finished ={}
for i in tqdm(range(len(similarities_finished))):
    similarities = similarities_finished.iloc[i]
    reversed_similarities = similarities[::-1]
    for j in range(len(reversed_similarities)):
        distance_to_series_finished[j] = distance_to_series_finished.get(j,[]) + [reversed_similarities[j]]

print(len(distance_to_series_finished))

distance_to_series_unfinished ={}
similarities_unfinished = similarities_unfinished['OpenAI']
for i in tqdm(range(len(similarities_unfinished))):
    similarities = similarities_unfinished.iloc[i]
    reversed_similarities = similarities[::-1]
    for j in range(len(reversed_similarities)):
        distance_to_series_unfinished[j] = distance_to_series_unfinished.get(j,[]) + [reversed_similarities[j]]

print(distance_to_series_unfinished)


l_finished=[]
for i in range(len(distance_to_series_finished)):
    l_finished.append(distance_to_series_finished[i])

l_unfinished=[]
for i in range(len(distance_to_series_unfinished)):
    l_unfinished.append(distance_to_series_unfinished[i])

In [ ]:



# Calculate means and standard deviations
means_finished = [np.mean(sublist) for sublist in l_finished]
medians_finished = [np.median(sublist) for sublist in l_finished]
stds_finished = [np.std(sublist) for sublist in l_finished]

means_unfinished = [np.mean(sublist) for sublist in l_unfinished]
medians_unfinished = [np.median(sublist) for sublist in l_unfinished]
stds_unfinished = [np.std(sublist) for sublist in l_unfinished]


# Create x-axis values (distances)
x_finished = range(len(means_finished)-1, -1, -1)  # Reversed range
x_unfinished = range(len(means_unfinished)-1, -1, -1)  # Reversed range

# Create figure and axis
plt.figure(figsize=(15, 8))

# Plot means with standard deviation areas
plt.plot(x_finished, means_finished, color='#1f77b4', label='Mean (Finished)', linewidth=2)
plt.plot(x_finished, medians_finished, color='#ff7f0e', label='Median (Finished)', linewidth=2)
plt.fill_between(x_finished, 
                 [m - s for m,s in zip(means_finished, stds_finished)],
                 [m + s for m,s in zip(means_finished, stds_finished)],
                 color='#1f77b4', alpha=0.2)

plt.plot(x_unfinished, means_unfinished, color='#2ca02c', label='Mean (Unfinished)', linewidth=2)
plt.plot(x_unfinished, medians_unfinished, color='#d62728', label='Median (Unfinished)', linewidth=2)
plt.fill_between(x_unfinished, 
                 [m - s for m,s in zip(means_unfinished, stds_unfinished)],
                 [m + s for m,s in zip(means_unfinished, stds_unfinished)],
                 color='#2ca02c', alpha=0.2)

plt.xlabel('Distance from End', fontsize=12)
plt.ylabel('Average Similarity Score', fontsize=12)
plt.ylim(0,1)

plt.title('Mean and Median Similarity Score vs Distance from End (with Standard Deviation)', fontsize=14)

# Add grid for better readability
plt.grid(True, linestyle='--', alpha=0.7)

# Add legend
plt.legend(fontsize=10)


plt.tight_layout()
plt.show()


In [ ]:
chosen_links_df =pd.read_csv('data/chosen_links_rank.csv')


rank_data = chosen_links_df['RankChosen'].dropna()

# Count the frequency of each rank
rank_counts = rank_data.value_counts()

plt.figure(figsize=(10, 6))
plt.bar(rank_counts.index, rank_counts.values, color='skyblue')
plt.xlabel('Position of Chosen Link in Article')
plt.xlim(1, 30)  # Limit to first 20 positions
plt.ylabel('Frequency')
plt.title('Distribution of Chosen Links Based on Position in Article')

# Set x-ticks to show every position within the limited range
plt.xticks(range(1, 31))  # Adjust as necessary

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [49]:
# Step 1: Replace -1 values with np.nan to ignore them during normalization
distance_matrix_normalized = np.where(distance_matrix == -1, np.nan, distance_matrix)

# Step 2: Perform normalization on the matrix, excluding NaN values
min_value = np.nanmin(distance_matrix_normalized)
max_value = np.nanmax(distance_matrix_normalized)
distance_matrix_normalized = (distance_matrix_normalized - min_value) / (max_value - min_value)

# Step 3: Replace NaNs (original -1 values) with 1
distance_matrix_normalized = np.where(np.isnan(distance_matrix_normalized), 1, distance_matrix_normalized)


In [ ]:
min_value = np.nanmin(distance_matrix_normalized)
max_value = np.nanmax(distance_matrix_normalized)
distance_matrix_normalized = (distance_matrix_normalized - min_value) / (max_value - min_value)

# Step 3: Replace NaNs (original -1 values) with 1
distance_matrix_normalized = np.where(np.isnan(distance_matrix_normalized), 1, distance_matrix_normalized)

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(distance_matrix_normalized, cmap="YlGnBu", annot=False, cbar=True)
plt.title('Heatmap of Normalized Distance Matrix')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.show()
